In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import datetime
import shutil

import numpy as np
import pandas as pd

from library import network as nw
from library import common as cm

In [ ]:
%run setup.py

np.random.seed(RANDOM_SEED)

In [ ]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

In [ ]:
# Choose features
ori_fea_save = ['delta_bs', '1_over_sqrt_tau', 'vega_n']        

In [ ]:
# Here, we specify hyperparameters, which will be used for training at the next step.
hypers = {
    'nodes_per_layer': (30, 30),
    'reg_alpha': 1e-4,
    'lr': 1e-4,
    'epochs': 2,
    'outact': OUTACT
}

##### Train a network with given hypers

Here, we use a bit different structure as the simulation data.
In case of no permutation, there are only one ckp, history, plot folder.
In case of permuation, in each of ckp, history and plot, we have many permutation folders.

As for the pnl folder:
In case of no permutation, only one pnl file.
in case of permutations, many pnl files.
This is consistent as the regression folder.


In [ ]:
PERMUTE = False

for VIX in [False, True]:
    
    if VIX:
        ori_fea = ori_fea_save + ['fake_vix']
    else:
        ori_fea = ori_fea_save
        
    res_dir = f'{DATA_DIR}Result/FREQ={FREQ}_HALFMONEY={HALF_MONEY}_MINM={MIN_M}_MAXM={MAX_M}_MINTAU={MIN_TAU_Days}_Permute={PERMUTE}_VIX={VIX}_MISSING={FEED_MISSING}_WINDOW={date_window}_KickTmrZeroVolume={REMOVE_TMR_ZERO_VOLUME}/'
    sub_res = res_dir + 'Network/'
    
    sub_res_dir = {
        'ckp': sub_res + 'ckp/',
        'history': sub_res + 'history/',
        'pnl': sub_res + 'pnl/',
        'plot': sub_res + 'plot/'
    }
    for key, value in sub_res_dir.items():
        os.makedirs(value, exist_ok=True)
        
    if not TRAIN_BY_YOURSELF:
        # then load the provided checkpoints of ANN
        sub_res_dir['ckp'] = f'{CKP_DIR}Permute={PERMUTE}_VIX={VIX}/'

    use_fea = [x + '_t' for x in ori_fea] + ['cp_int']
    
    df_temp = df.copy()
    
    df_temp = nw.rolling_net(
        df_temp,
        ori_fea=ori_fea,
        use_fea=use_fea,
        end_period=max_period,
        hypers=hypers,
        sub_res_dir=sub_res_dir,
        train_by_yourself=TRAIN_BY_YOURSELF
    )

    cm.store_pnl(df_temp, df_temp['delta_nn'], sub_res_dir['pnl'] + 'pnl.csv')
    
del df

#### Permute

In [ ]:
PERMUTE = True
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

In [ ]:


for VIX in [False, True]:
    res_dir = f'{DATA_DIR}Result/FREQ={FREQ}_HALFMONEY={HALF_MONEY}_MINM={MIN_M}_MAXM={MAX_M}_MINTAU={MIN_TAU_Days}_Permute={PERMUTE}_VIX={VIX}_MISSING={FEED_MISSING}_WINDOW={date_window}_KickTmrZeroVolume={REMOVE_TMR_ZERO_VOLUME}/'
    sub_res = res_dir + 'Network/'
    
    if VIX:
        ori_fea = ori_fea_save + ['fake_vix']
    else:
        ori_fea = ori_fea_save
        
    for p in range(NUM_PERMUTE):
        """
        We keep all the pnl in the same folder, because each
        permute only has one pnl file and we can use the same code 
        for evaluation.
        """
        sub_res_dir = {
            'ckp': sub_res + f'ckp/permute{p}/',
            'history': sub_res + f'history/permute{p}/',
            'pnl': sub_res + f'pnl/',
            'plot': sub_res + f'plot/permute{p}/'
        }
        for key, value in sub_res_dir.items():
            os.makedirs(value, exist_ok=True)
        
        if not TRAIN_BY_YOURSELF:
            sub_res_dir['ckp'] = f'{CKP_DIR}Permute={PERMUTE}_VIX={VIX}/permute{p}/'
        
        use_fea = [x + '_t' for x in ori_fea] + ['cp_int']
        
        df_permute = df.copy()
        df_permute = cm.rolling_permute(df_permute, random_seed=p+100)
        
        df_permute = nw.rolling_net(
            df_permute, 
            ori_fea=ori_fea, 
            use_fea=use_fea,
            end_period=max_period,
            hypers=hypers, 
            sub_res_dir=sub_res_dir,
            train_by_yourself=TRAIN_BY_YOURSELF
        )
        
        cm.store_pnl(df_permute, df_permute['delta_nn'], sub_res_dir['pnl'] + f'pnl{p}.csv')